In [1]:
import dask.dataframe as dd
import pandas as pd                                   
import time

In [2]:
#Reading from a csv
ddf = dd.read_csv(r'D:\Projects\Dask_Dataframe\WeatherEvents\WeatherEvents.csv',dtype={"ZipCode": "Int64"})

In [3]:
df = pd.read_csv(r'D:\Projects\Dask_Dataframe\WeatherEvents\WeatherEvents.csv')

In [4]:
#Getting a sneak peak in the data
ddf.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149


In [5]:
#To get the datatypes of columns
ddf._meta.dtypes

EventId              string[pyarrow]
Type                 string[pyarrow]
Severity             string[pyarrow]
StartTime(UTC)       string[pyarrow]
EndTime(UTC)         string[pyarrow]
Precipitation(in)            float64
TimeZone             string[pyarrow]
AirportCode          string[pyarrow]
LocationLat                  float64
LocationLng                  float64
City                 string[pyarrow]
County               string[pyarrow]
State                string[pyarrow]
ZipCode                        Int64
dtype: object

In [6]:
ddf.describe()

,Precipitation(in),LocationLat,LocationLng,ZipCode
npartitions=1,,,,
,float64,float64,float64,Float64
,...,...,...,...


In [7]:
ddf

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
npartitions=16,,,,,,,,,,,,,,
,string,string,string,string,string,float64,string,string,float64,float64,string,string,string,Int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


<h3>Value_counts</h3>
<p>Counting the number of each unique value; & comparing dask VS. pandas dataframe time</p>

In [8]:
#Calculating the time of value_counts in dask
stime = time.time()
unique_value_count = ddf["County"].value_counts().compute()
etime = time.time()
print("Time :", etime - stime)

Time : 2.0748915672302246


In [9]:
#Calculating the time of value_counts in pandas
stime = time.time()
df["County"].value_counts()
etime = time.time()
print("Time :", etime - stime)

Time : 0.3565545082092285


In [10]:
ddf["Type"].value_counts().compute()

Type
Fog              2014190
Cold              232192
Storm              61317
Precipitation     157186
Hail                2927
Rain             5000794
Snow             1158575
Name: count, dtype: int64

<h3>Group by</h3>
<p>grouping by a specific county here "anderw", then grouping the records from andrew by type to know the frequent weather there.</br>
Comparing the time of dask vs pandas in performing these operations. 
Tried it using compute and persist to show the difference between them. </br>
Persist: Cache the answer in the memory  </br>
Compute: compute the result and return it as a pandas item</p>

In [11]:
#Using Compute Group by
stime = time.time()
ddf_andrews = ddf[ddf["County"] == "Andrews"]
res_snow = ddf_andrews.groupby("Type")["EventId"].count().compute()
print(time.time() - stime)
print(res_snow)
print(type(res_snow))

5.231825351715088
Type
Fog               384
Precipitation      33
Rain             1571
Cold               59
Snow               57
Storm              21
Name: EventId, dtype: int64
<class 'pandas.core.series.Series'>


In [12]:
#Pandas Grouby
stime = time.time()
df_andrews = df[df["County"] == "Andrews"]
res_snow = df_andrews.groupby("Type")["EventId"].count()
print(time.time() - stime)
print(res_snow)

0.4819455146789551
Type
Cold               59
Fog               384
Precipitation      33
Rain             1571
Snow               57
Storm              21
Name: EventId, dtype: int64


In [13]:
#Using Persist
ddf_andrews = ddf[ddf["County"] == "Andrews"].persist()
res_snow = ddf_andrews.groupby("Type")["EventId"].count().compute()
print(res_snow)
print(type(ddf_andrews))

Type
Fog               384
Precipitation      33
Rain             1571
Cold               59
Snow               57
Storm              21
Name: EventId, dtype: int64
<class 'dask_expr._collection.DataFrame'>


<table>
    <tr>
    <th></th>
    <th>Compute</th>
    <th>Persist</th>
    </tr>
    <tr>
        <td>Execution</td>
        <td>It computes the dask graph and return the result</td>
        <td>computes the dask graph also, but return a refrence to the in memory result</td>
    </tr>
    <tr>
        <td>Memory</td>
        <td>It returns the result in the form of pandas series, pandas dataframe..etc</td>
        <td>The result is saved in the</td>
    </tr>
    <tr>
        <td>Use Case</td>
        <td>When the final result needed or want it in another format pandas dataframe</td>
        <td>When the result needed to be cached to make further computations on</td>
    </tr>
</table>

<h3></h3>

<h3>Partitions</h3>
<p>Taking a subset of the dataframe partitons using partitions </br>
and viewing the number of partitions using npartitions</p>

In [14]:
#Taking only two partitions of the data
sliced_data = ddf.partitions[3:5]
print("Sliced data no of partitions: ",sliced_data.npartitions)
print("Original data no of partitions: ",ddf.npartitions)
print("andrews data no of partitions: ",ddf.npartitions)

Sliced data no of partitions:  2
Original data no of partitions:  16
andrews data no of partitions:  16


<h3>repartition</h3>
<p>Repartitioning andrew dataframe to 17 partition using npartition</p>

In [15]:
#Repartition andrews data
ddf_andrews = ddf_andrews.repartition(npartitions=17)
print(ddf_andrews.npartitions)

17


<h3>Applying function to big data dask vs pandas</h3>
<p>Used map_partition in dask to apply function 'split_date' here to every partition in the dataframe. </br>
Used apply in pandas to apply the same function to the whole dataframe.</p>

In [16]:
#Defining the structure of the returned dataframe
meta = {
    'EventId': 'string',                 
    'Type': 'string',                     
    'Severity': 'string',                 
    'StartTime(UTC)': 'string',          
    'EndTime(UTC)': 'string',             
    'Precipitation(in)': 'float64',       
    'TimeZone': 'string',                
    'AirportCode': 'string',             
    'LocationLat': 'float64',            
    'LocationLng': 'float64',            
    'City': 'string',                    
    'County': 'string',                  
    'State': 'string',                   
    'ZipCode': 'Int64',
    'StartDate': "string",
    'StartTime': "string",
    'EndDate': "string",
    'EndTime': "string"
}

In [17]:
#Applying a function to all data partitions
#It took long time but was able to do it
stime = time.time()

#Split_date function to split starttime and endtime to date values and time values
def split_date(ddf):
    ddf[["StartDate","StartTime"]] = ddf["StartTime(UTC)"].str.split(' ',expand=True)
    ddf[["EndDate","EndTime"]] = ddf["EndTime(UTC)"].str.split(' ',expand=True)
    return ddf

#Applying split_date function to every partition in the dataframe
ddf2 = ddf.map_partitions(split_date, meta=meta).persist()
print(time.time() - stime)

58.253225803375244


In [18]:
ddf2.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,StartDate,StartTime,EndDate,EndTime
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-06,23:14:00,2016-01-07,00:34:00
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-07,04:14:00,2016-01-07,04:54:00
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-07,05:54:00,2016-01-07,15:34:00
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-08,05:34:00,2016-01-08,05:54:00
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-08,13:54:00,2016-01-08,15:54:00


In [19]:
#Spliting date function to be used by pandas dataframe
def split_date(df):
    df[["StartDate","StartTime"]] = df["StartTime(UTC)"].split(' ',expand=True)
    df[["EndDate","EndTime"]] = df["EndTime(UTC)"].split(' ',expand=True)
    return df

In [20]:
#Changing type of starttime and endtime to string to be able to split them
df.astype({'StartTime(UTC)': 'string[pyarrow]',"EndTime(UTC)": "string[pyarrow]"}).dtypes

EventId                       object
Type                          object
Severity                      object
StartTime(UTC)       string[pyarrow]
EndTime(UTC)         string[pyarrow]
Precipitation(in)            float64
TimeZone                      object
AirportCode                   object
LocationLat                  float64
LocationLng                  float64
City                          object
County                        object
State                         object
ZipCode                      float64
dtype: object

In [94]:
#using apply to apply a function on big data may cause unable to locate memory
stime = time.time()
df2 = df.apply(split_date,axis=1)
print(time.time() - stime)
df2.head()

MemoryError: Unable to allocate 658. MiB for an array with shape (10, 8627181) and data type object

<h3>loc dask vs pandas</h3>
<p>Comparing the time taken between dask vs pandas dataframe to filter to specific column using loc</p>

In [21]:
stime = time.time()
print(ddf2.loc[:,["Severity","Type"]].compute())
print(time.time() - stime)

        Severity  Type
0          Light  Snow
1          Light  Snow
2          Light  Snow
3          Light  Snow
4          Light  Snow
...          ...   ...
535912     Light  Snow
535913  Moderate  Snow
535914    Severe  Cold
535915    Severe  Cold
535916    Severe  Cold

[8627181 rows x 2 columns]
0.27005910873413086


In [22]:
stime = time.time()
print(df.loc[:,["Severity","Type"]])
print(time.time() - stime)

         Severity  Type
0           Light  Snow
1           Light  Snow
2           Light  Snow
3           Light  Snow
4           Light  Snow
...           ...   ...
8627176     Light  Snow
8627177  Moderate  Snow
8627178    Severe  Cold
8627179    Severe  Cold
8627180    Severe  Cold

[8627181 rows x 2 columns]
0.15075087547302246


<h3>drop_duplicates dask vs pandas</h3>
<p>Comparing the time taken between dask and pandas dataframe to drop duplicates</p>

In [23]:
#Dask
stime = time.time()
ddf.drop_duplicates()
print(time.time() - stime)

0.03974199295043945


In [24]:
#Pandas
stime = time.time()
df.drop_duplicates()
print(time.time() - stime)

17.120985746383667


<h3>Replace dask vs pandas</h3>
<p>Comparing the time taken between dask vs pandas to replace snow value to ice</p>

In [25]:
#dask
stime = time.time()
ddf["Type"] =ddf["Type"].replace({"Snow" : "Ice"})
print(time.time() - stime)
ddf.head()

0.016324520111083984


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Ice,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
1,W-2,Ice,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
2,W-3,Ice,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
3,W-4,Ice,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
4,W-5,Ice,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149


In [26]:
#pandas
stime = time.time()
df["Type"] =df["Type"].replace({"Snow" : "Ice"})
print(time.time() - stime)
df.head()

0.6524040699005127


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Ice,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Ice,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Ice,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Ice,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Ice,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


<h3>Drop vs Pop vs Dropna in dask</h3>

In [27]:
#copying the dataframe to new one as not to affect the original
ddf_copy = ddf.copy()

In [29]:
#dropping using drop
stime = time.time()
eventId = ddf_copy.drop(columns="EventId")
print(time.time() - stime)

0.027615070343017578


In [30]:
#Drop returns a new dataframe without the dropped column
print(type(eventId))
eventId.head()

<class 'dask_expr._collection.DataFrame'>


,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,Ice,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
1,Ice,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
2,Ice,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
3,Ice,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
4,Ice,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149


In [39]:
#Original dataframe is not modified
ddf_copy.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Ice,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
1,W-2,Ice,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
2,W-3,Ice,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
3,W-4,Ice,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
4,W-5,Ice,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149


In [46]:
ddf_copy2 = ddf.copy()

In [47]:
#Drop using pop
eventid = ddf_copy2.pop("EventId").compute()

In [48]:
#pop returns the dropped column
print(type(eventid))
eventid

<class 'pandas.core.series.Series'>


0               W-1
1               W-2
2               W-3
3               W-4
4               W-5
            ...    
535912    W-9091890
535913    W-9091891
535914    W-9091892
535915    W-9091893
535916    W-9091894
Name: EventId, Length: 8627181, dtype: object

In [49]:
#It modifies the dataframe
ddf_copy2.head()

,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,Ice,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
1,Ice,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
2,Ice,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
3,Ice,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
4,Ice,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149


<table>
  <tr>
    <th>Drop</th>
    <th>Pop</th>
    <th>Dropna</th>
  </tr>
  <tr>
    <td>Dropping a column and return a dataframe without it</td>
    <td>Dropping a column and returns it and and modifies the dataframe</td>
    <td>Dropping null values</td>
  </tr>
</table>

<h3>Assign vs Eval in Dask</h3>
<p>There is no insert function in dask as in pandas</p>

In [50]:
ddf = ddf.assign(new_col= "new" )

In [51]:
ddf.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,new_col
0,W-1,Ice,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new
1,W-2,Ice,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new
2,W-3,Ice,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new
3,W-4,Ice,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new
4,W-5,Ice,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new


In [52]:
stime = time.time()
ddf_tmp = ddf.assign(new_col2= ddf["Type"] + " " + ddf["Severity"])
print(time.time() - stime)
ddf_tmp.head()

0.03738594055175781


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,new_col,new_col2
0,W-1,Ice,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new,Ice Light
1,W-2,Ice,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new,Ice Light
2,W-3,Ice,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new,Ice Light
3,W-4,Ice,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new,Ice Light
4,W-5,Ice,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new,Ice Light


In [53]:
ddf2.eval('new_column = Type + Severity').compute()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,StartDate,StartTime,EndDate,EndTime,new_column
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-06,23:14:00,2016-01-07,00:34:00,SnowLight
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-07,04:14:00,2016-01-07,04:54:00,SnowLight
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-07,05:54:00,2016-01-07,15:34:00,SnowLight
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-08,05:34:00,2016-01-08,05:54:00,SnowLight
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-08,13:54:00,2016-01-08,15:54:00,SnowLight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535912,W-9091890,Snow,Light,2022-12-21 23:00:00,2022-12-21 23:42:00,0.01,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520,2022-12-21,23:00:00,2022-12-21,23:42:00,SnowLight
535913,W-9091891,Snow,Moderate,2022-12-21 23:42:00,2022-12-21 23:53:00,0.04,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520,2022-12-21,23:42:00,2022-12-21,23:53:00,SnowModerate
535914,W-9091892,Cold,Severe,2022-12-21 23:53:00,2022-12-24 02:53:00,0.00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520,2022-12-21,23:53:00,2022-12-24,02:53:00,ColdSevere
535915,W-9091893,Cold,Severe,2022-12-24 03:53:00,2022-12-24 07:53:00,0.00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520,2022-12-24,03:53:00,2022-12-24,07:53:00,ColdSevere


<table>
  <tr>
    <th>Assign</th>
    <th>Eval</th>
  </tr>
  <tr>
    <td>It can make implement complex functions to insert new columns</td>
    <td>It Evaluates a string to insert new column</td>
  </tr>
</table>

<h3>Sort_value pandas vs dask</h3>
<p>Comparing the time taken between dask vs pandas to sort values using sort_values</p>

In [54]:
#Changing the type of precipitation column to int to sort it
ddf.astype({"Precipitation(in)": "Int64"}).dtypes

EventId              string[pyarrow]
Type                 string[pyarrow]
Severity             string[pyarrow]
StartTime(UTC)       string[pyarrow]
EndTime(UTC)         string[pyarrow]
Precipitation(in)              Int64
TimeZone             string[pyarrow]
AirportCode          string[pyarrow]
LocationLat                  float64
LocationLng                  float64
City                 string[pyarrow]
County               string[pyarrow]
State                string[pyarrow]
ZipCode                        Int64
new_col                       object
dtype: object

In [55]:
stime = time.time()
ddf_sorted = ddf.sort_values(by="Precipitation(in)", ascending=False).compute()
print(time.time() - stime)
ddf_sorted.head()

26.67107605934143


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,new_col
365804,W-3239430,Precipitation,UNK,2016-01-17 15:58:00,2016-01-19 15:58:00,1104.13,US/Eastern,KMCF,27.8493,-82.5212,Tampa,Hillsborough,FL,33621,new
216295,W-7053024,Precipitation,UNK,2016-04-24 21:35:00,2016-05-23 04:55:00,668.58,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,new
216263,W-7052992,Precipitation,UNK,2016-02-08 06:55:00,2016-03-02 20:15:00,557.04,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,new
216269,W-7052998,Precipitation,UNK,2016-03-15 04:15:00,2016-04-06 02:15:00,515.79,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,new
216247,W-7052976,Precipitation,UNK,2016-01-17 08:35:00,2016-02-07 14:15:00,497.97,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,new


In [56]:
stime = time.time()
df_sorted = df.sort_values(by="Precipitation(in)", ascending=False)
print(time.time() - stime)
df_sorted.head()

16.43259310722351


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
3070926,W-3239430,Precipitation,UNK,2016-01-17 15:58:00,2016-01-19 15:58:00,1104.13,US/Eastern,KMCF,27.8493,-82.5212,Tampa,Hillsborough,FL,33621.0
6691279,W-7053024,Precipitation,UNK,2016-04-24 21:35:00,2016-05-23 04:55:00,668.58,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341.0
6691247,W-7052992,Precipitation,UNK,2016-02-08 06:55:00,2016-03-02 20:15:00,557.04,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341.0
6691253,W-7052998,Precipitation,UNK,2016-03-15 04:15:00,2016-04-06 02:15:00,515.79,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341.0
6691231,W-7052976,Precipitation,UNK,2016-01-17 08:35:00,2016-02-07 14:15:00,497.97,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341.0


<h3>Fillna in dask</h3>

In [57]:
ddf2["Type"].fillna(ddf2["Type"].mode).compute()

0         Snow
1         Snow
2         Snow
3         Snow
4         Snow
          ... 
535912    Snow
535913    Snow
535914    Cold
535915    Cold
535916    Cold
Name: Type, Length: 8627181, dtype: object

<h3>Filtering in dask</h3>
<p>There is no filter function in dask as it is in pandas so we can filter using query or loc as was mentioned above</p>

In [58]:
#Filtering rows that have precipitation higher than 500
ddf2.query(" `Precipitation(in)` > 500").sort_values(by="Precipitation(in)").compute()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,StartDate,StartTime,EndDate,EndTime
216269,W-7052998,Precipitation,UNK,2016-03-15 04:15:00,2016-04-06 02:15:00,515.79,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-03-15,04:15:00,2016-04-06,02:15:00
216263,W-7052992,Precipitation,UNK,2016-02-08 06:55:00,2016-03-02 20:15:00,557.04,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-02-08,06:55:00,2016-03-02,20:15:00
216295,W-7053024,Precipitation,UNK,2016-04-24 21:35:00,2016-05-23 04:55:00,668.58,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-04-24,21:35:00,2016-05-23,04:55:00
365804,W-3239430,Precipitation,UNK,2016-01-17 15:58:00,2016-01-19 15:58:00,1104.13,US/Eastern,KMCF,27.8493,-82.5212,Tampa,Hillsborough,FL,33621,2016-01-17,15:58:00,2016-01-19,15:58:00


In [59]:
#Filtering rows that have precipitation higher than 500 with external value
value = 500
ddf2.query(f"`Precipitation(in)` > {value}").sort_values(by="Precipitation(in)").compute()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,StartDate,StartTime,EndDate,EndTime
216269,W-7052998,Precipitation,UNK,2016-03-15 04:15:00,2016-04-06 02:15:00,515.79,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-03-15,04:15:00,2016-04-06,02:15:00
216263,W-7052992,Precipitation,UNK,2016-02-08 06:55:00,2016-03-02 20:15:00,557.04,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-02-08,06:55:00,2016-03-02,20:15:00
216295,W-7053024,Precipitation,UNK,2016-04-24 21:35:00,2016-05-23 04:55:00,668.58,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-04-24,21:35:00,2016-05-23,04:55:00
365804,W-3239430,Precipitation,UNK,2016-01-17 15:58:00,2016-01-19 15:58:00,1104.13,US/Eastern,KMCF,27.8493,-82.5212,Tampa,Hillsborough,FL,33621,2016-01-17,15:58:00,2016-01-19,15:58:00


In [60]:
ddf2.sort_values(by="Precipitation(in)").compute()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,StartDate,StartTime,EndDate,EndTime
12584,W-1727673,Rain,Light,2017-09-25 05:26:00,2017-09-25 05:44:00,0.00,US/Central,KBFW,47.2521,-91.4098,Two Harbors,Lake,MN,55616,2017-09-25,05:26:00,2017-09-25,05:44:00
301812,W-7142979,Fog,Severe,2022-03-29 10:27:00,2022-03-29 10:37:00,0.00,US/Eastern,KTLH,30.3935,-84.3513,Tallahassee,Leon,FL,32310,2022-03-29,10:27:00,2022-03-29,10:37:00
301811,W-7142978,Fog,Severe,2022-03-29 09:28:00,2022-03-29 09:53:00,0.00,US/Eastern,KTLH,30.3935,-84.3513,Tallahassee,Leon,FL,32310,2022-03-29,09:28:00,2022-03-29,09:53:00
301810,W-7142977,Fog,Severe,2022-03-29 08:47:00,2022-03-29 09:20:00,0.00,US/Eastern,KTLH,30.3935,-84.3513,Tallahassee,Leon,FL,32310,2022-03-29,08:47:00,2022-03-29,09:20:00
31,W-32,Snow,Light,2016-01-31 07:55:00,2016-01-31 08:35:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-31,07:55:00,2016-01-31,08:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216247,W-7052976,Precipitation,UNK,2016-01-17 08:35:00,2016-02-07 14:15:00,497.97,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-01-17,08:35:00,2016-02-07,14:15:00
216269,W-7052998,Precipitation,UNK,2016-03-15 04:15:00,2016-04-06 02:15:00,515.79,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-03-15,04:15:00,2016-04-06,02:15:00
216263,W-7052992,Precipitation,UNK,2016-02-08 06:55:00,2016-03-02 20:15:00,557.04,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-02-08,06:55:00,2016-03-02,20:15:00
216295,W-7053024,Precipitation,UNK,2016-04-24 21:35:00,2016-05-23 04:55:00,668.58,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-04-24,21:35:00,2016-05-23,04:55:00


<h3>Pandas vs Dask Comparison</h3>
<table>
  <tr>
    <th></th>
    <th>Pandas</th>
    <th>Dask</th>
  </tr>
  <tr>
    <td>Small Dataset</td>
    <td>It will work faster as it works on the whole dataset at a time</td>
    <td>Not needed since the data is small, and as it works with partitions it will take more time</td>
  </tr>
  <tr>
    <td>Large dataset (>RAM size)</td>
    <td>Pandas will fail in speed or fail or won't be able to hold the data</td>
    <td>It will be faster and more efficient as it works in parallel using partition</td>
  </tr>
  <tr>
    <td>Methods like join, groupby, sorting..etc</td>
    <td>Fast</td>
    <td>Slow as methods that requires lots of partition reordering slows down the process</td>
  </tr>
  <tr>
    <td>Distributed data on different computers</td>
    <td>can't work with distributed data alone</td>
    <td>can work with distributed data</td>
  </tr>
</table>